# Distilling Knowledge in Multiple Students Using Generative Models

In [49]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [51]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(teacher.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_29 (Dense)             (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

In [52]:
teacher.load_weights("Teacher_FMNIST_92.h5")

In [ ]:
# Train the teacher model as usual
epochs = 1
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))


In [3]:
#checking teachers test accuracy
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255 
Y_test = np_utils.to_categorical(y_test, nb_classes)

teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.4480 - accuracy: 0.9211


[0.44798123836517334, 0.9211000204086304]

In [5]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)



# 1 Student case
# ---------------------------------------------
s1Train=train_dense[:,:256]


## GANs' Training

In [6]:
# import np.random import random
BATCH_SIZE=32
def smooth_real_labels(y):
    return y - 0.3+(np.random.random(y.shape)*0.5)
def smooth_fake_labels(y):
    return y + (0.3 * np.random.random(y.shape))
def build_gan(gen,disc): 
    disc.trainable = False
    input= Input(shape=input_shape)
    output = gen(input)
    output2= disc(output)
    gan=Model(input,output2)

    gan.compile(Adam(lr=0.0002),loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return gan

In [27]:
def build_sdiscriminator():
    
    input2 = Input(shape=(256,),name='input')
    inp=Dense(64,use_bias=False)(input2)

    leaky_relu = LeakyReLU(alpha=0.2)(inp)
    
    conv3 = Dense(128)(leaky_relu)
    b_n = BatchNormalization()(conv3)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(256)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(512)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)
    conv4 = Dense(1024)(leaky_relu)
    b_n = BatchNormalization()(conv4)
    leaky_relu = LeakyReLU(alpha=0.2)(b_n)

    dense = Dense(1,activation='sigmoid',name='dense')(leaky_relu)
    output2=Dense(256)(leaky_relu)
    disc = Model(input2,[dense,output2])          
    disc.compile(optd,loss=['binary_crossentropy','mse'],metrics=['accuracy'])

    return disc

In [28]:
optd = Adam(lr=0.0002)
opt = Adam(lr=0.0002)

In [37]:
def build_sgenerator(name):

    student1 = Sequential()
    student1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(64, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(256,name='req'+name))

    student1.compile(opt,loss='mse',metrics=['accuracy'])
    student1.summary()
    return student1



In [38]:
def training(generator,discriminator,gan,features,epo=20):
    BATCH_SIZE = 128
    discriminator.trainable = True
    total_size = X_train.shape[0]
    indices = np.arange(0,total_size ,BATCH_SIZE)
    all_disc_loss = []
    all_gen_loss = []
    all_class_loss=[]
    if total_size % BATCH_SIZE:
        indices = indices[:-1]
    for e in range(epo):
        
        progress_bar = Progbar(target=len(indices))
        np.random.shuffle(indices)
        epoch_gen_loss = []
        epoch_disc_loss = []
        epoch_class_loss= []
        for i,index in enumerate(indices):
        
            inputs=X_train[index:index+BATCH_SIZE]
            strain = features[index:index+BATCH_SIZE]
            y_real = np.ones((BATCH_SIZE,1))
            y_fake = np.zeros((BATCH_SIZE,1))

            #Generator Training
            fake_images = generator.predict_on_batch(inputs)

            #Disrciminator Training
            disc_real_loss1,_,disc_real_loss2,_,_= discriminator.train_on_batch(strain,[y_real,strain])
            disc_fake_loss1,_,disc_fake_loss2,_,_= discriminator.train_on_batch(fake_images,[y_fake,strain])

            #Gans Training
            discriminator.trainable = False
            gan_loss,_,gan_loss2,_,_ = gan.train_on_batch(inputs, [y_real,strain])

            discriminator.trainable = True

            disc_loss = (disc_fake_loss1 + disc_real_loss1)/2
            epoch_disc_loss.append(disc_loss)
            progress_bar.update(i+1)

            epoch_gen_loss.append((gan_loss))

        avg_epoch_disc_loss = np.array(epoch_disc_loss).mean()
        avg_epoch_gen_loss = np.array(epoch_gen_loss).mean()
        all_disc_loss.append(avg_epoch_disc_loss)
        all_gen_loss.append(avg_epoch_gen_loss)
        print("Epoch: %d | Discriminator Loss: %f | Generator Loss: %f | " % (e+1,avg_epoch_disc_loss,avg_epoch_gen_loss))

    return generator

In [39]:
discriminator1 = build_sdiscriminator()
# discriminator2 = build_sdiscriminator()
# discriminator3 = build_sdiscriminator()
# discriminator4 = build_sdiscriminator()

In [40]:
s1=build_sgenerator("s1")
# s2=build_sgenerator('s2')
# s3=build_sgenerator("s3")
# s4=build_sgenerator('s4')

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 10, 10, 16)        4624      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 4, 4, 16)         

In [41]:
gan1 = build_gan(s1,discriminator1)
# gan2 = build_gan(s2,discriminator2)
# gan3 = build_gan(s3,discriminator3)
# gan4 = build_gan(s4,discriminator4)

In [42]:
s1 = training(s1,discriminator1,gan1,s1Train,epo=55)
# s2 = training(s2,discriminator2,gan2,s2Train,epo=55)
# s3 = training(s3,discriminator3,gan3,s3Train,epo=60)
# s4 = training(s4,discriminator4,gan4,s4Train,epo=58)

375/375 [==============================] - 15s 37ms/step
Epoch: 1 | Discriminator Loss: 0.560325 | Generator Loss: 1.592662 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 2 | Discriminator Loss: 0.291414 | Generator Loss: 0.926031 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 3 | Discriminator Loss: 0.249491 | Generator Loss: 0.876279 | 
375/375 [==============================] - 14s 37ms/step
Epoch: 4 | Discriminator Loss: 0.231559 | Generator Loss: 0.874988 | 
375/375 [==============================] - 14s 36ms/step
Epoch: 5 | Discriminator Loss: 0.219904 | Generator Loss: 0.805115 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 6 | Discriminator Loss: 0.209813 | Generator Loss: 0.746718 | 
375/375 [==============================] - 14s 38ms/step
Epoch: 7 | Discriminator Loss: 0.200672 | Generator Loss: 0.716862 | 
375/375 [==============================] - 13s 35ms/step
Epoch: 8 | Discriminator Loss: 0.194452 | Generator Lo

### **1 Student**

In [43]:
o1=s1.get_layer("reqs1").output
output=Activation('relu')(o1)
output2=Dropout(0.5)(output)
output3=Dense(10,activation="softmax", name="d1")(output2)
mm1=Model(s1.get_layer('s1').input, output3)
my_weights=teacher.get_layer('dense_2').get_weights()

In [44]:
mm1.get_layer('d1').set_weights(my_weights)

In [45]:
for l in mm1.layers[:len(mm1.layers)-1]:
    l.trainable=False


In [46]:
mm1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [47]:
batch_size = 128
mm1.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=50,
          verbose=1, validation_data=(X_val, Y_val))

Epoch 1/50
375/375 [==============================] - 2s 5ms/step - loss: 3.4529 - accuracy: 0.5112 - val_loss: 0.4900 - val_accuracy: 0.8622
Epoch 2/50
375/375 [==============================] - 2s 4ms/step - loss: 1.0235 - accuracy: 0.7492 - val_loss: 0.4219 - val_accuracy: 0.8658
Epoch 3/50
375/375 [==============================] - 1s 2ms/step - loss: 0.7258 - accuracy: 0.7856 - val_loss: 0.3987 - val_accuracy: 0.8646
Epoch 4/50
375/375 [==============================] - 2s 4ms/step - loss: 0.6132 - accuracy: 0.7982 - val_loss: 0.3845 - val_accuracy: 0.8691
Epoch 5/50
375/375 [==============================] - 1s 3ms/step - loss: 0.5478 - accuracy: 0.8124 - val_loss: 0.3794 - val_accuracy: 0.8677
Epoch 6/50
375/375 [==============================] - 1s 4ms/step - loss: 0.5394 - accuracy: 0.8131 - val_loss: 0.3762 - val_accuracy: 0.8711
Epoch 7/50
375/375 [==============================] - 1s 4ms/step - loss: 0.5086 - accuracy: 0.8188 - val_loss: 0.3708 - val_accuracy: 0.8690
Epoch 

In [48]:
loss, acc = mm1.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.3951 - accuracy: 0.8588
